# DoubleDrift

#### Project description:
_Adaptation of experiment 1 (perception) of Lisa & Cavanagh, 2015, Current Biology<br>
(http://dx.doi.org/10.1016/j.cub.2015.08.021) for the AMU Neuroscience Master APP 2024 courses._

#### Hypothesis: 
_Participants mislocalize perceptively the drifting gabor but saccade to correctly to its<br>
 physical position_
 
#### Exercice for APP2024:
_Analyse data to reach an adapted version of Figure 1 of [Lisa & Cavanagh, 2015, Current Biology](http://dx.doi.org/10.1016/j.cub.2015.08.021)_

<img src="img/Lisi_Cavanagh_2015_CB_Figure1.png" width=700 alt="Figure 1">
<!-- ![Lisi_Cavanagh_2015_CB_Figure1.png]()  -->

#### Eye movement data analysis:

- [x] Step 1. Extract time series
- [x] Step 2. Extract saccades for each trials

#### Step 1: extract time series

In [1]:
# Imports
import os
import numpy as np
import glob
import pandas as pd
import itertools
import scipy.io
from sac_utils import vecvel, microsacc_merge, saccpar, isincircle
import ipdb

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

In [2]:
# Define folders
base_dir = '..'
data_dir = '{}/data'.format(base_dir)
subject = 'sub-06'
sessions = ['ses-02']
subject_num = subject[4:]
fig_dir = '{}/{}/figures'.format(data_dir, subject)

In [3]:
# Define data filenames
event_filenames = []
eyetrack_filnames = []
mat_filenames = []

for session in sessions:
    event_filenames.append(sorted(glob.glob('{}/{}/{}/beh/*.tsv'.format(data_dir, subject, session))))
    eyetrack_filnames.append(sorted(glob.glob('{}/{}/{}/beh/*.edf'.format(data_dir, subject, session))))
    mat_filenames.append(sorted(glob.glob('{}/{}/{}/beh/*_matlab.mat'.format(data_dir, subject, session))))
event_filenames = list(itertools.chain(*event_filenames))
eyetrack_filnames = list(itertools.chain(*eyetrack_filnames))
mat_filenames = list(itertools.chain(*mat_filenames))
num_run = len(event_filenames)

In [4]:
# Create message and data files
for eyetrack_file in eyetrack_filnames:
    
    if not os.path.exists(eyetrack_file.replace('.edf','.msg')):
        os.system('edf2asc {} -e -y'.format(eyetrack_file))
        os.rename(eyetrack_file.replace('.edf','.asc'),eyetrack_file.replace('.edf','.msg'))

    if not os.path.exists(eyetrack_file.replace('.edf','.dat')):
        os.system('edf2asc {} -s -miss -1.0 -y'.format(eyetrack_file))
        os.rename(eyetrack_file.replace('.edf','.asc'),eyetrack_file.replace('.edf','.dat'))


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 4.2.1.0 Linux   standalone Jun 18 2021 
(c)1995-2021 by SR Research, last modified Jun 18 2021

processing file ../data/sub-06/ses-02/beh/sub-06_ses-02_task-DoubleDriftSaccade_run-01_eyetrack.edf 
loadEvents = 1
Preamble of file ../data/sub-06/ses-02/beh/sub-06_ses-02_task-DoubleDriftSaccade_run-01_eyetrack.edf
| DATE: Fri Sep 13 08:55:23 2013                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v5.12 May 12 2017                                             |
| CAMERA: Eyelink GL Version 1.2 Sensor=AI7                                   |
| SERIAL NUMBER: CLG-BAF38                                                    |
| CAMERA_CONFIG: BAF38200.SCD                       

In [5]:
# Collect MSG data
msg_outputs = ['trial_onset', 'trial_offset', 'button_press_onset', 'button_left', 'button_right', 
               'fix_break', 'motion_onset', 'motion_offset', 'fixation_onset', 'fixation_offset', 
               'response_onset', 'response_offset']
num_trials = 100  # number of trials per run
 
for msg_output in msg_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(msg_output))

t_run = 0
for eyetrack_file in eyetrack_filnames:
    
    msgfid = open(eyetrack_file.replace('.edf','.msg'))
    first_last_time, first_time, last_time = False, False, False

    while not first_last_time:
        line_read = msgfid.readline()

        if not line_read == '':
            la = line_read.split()

            if len(la) > 2:
                if la[2] == 'RECORD_START' and not first_time: 
                    first_time = True
                if la[2] == 'RECORD_STOP' and not last_time:
                    last_time = True
            if len(la) > 4:
                if la[2] == 'trial' and la[4]=='check':
                    # trial %i check fixation at %f
                    trial_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[4]=='ended':
                    # trial %i ended
                    trial_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[4]=='fixation':
                    # trial %i fixation break at %f
                    fix_break[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'motion_onset':
                    # motion_onset %i at %f
                    motion_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'motion_offset':
                    # motion_offset %i at %f
                    motion_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'fixation_onset':
                    # fixation_onset %i at %f
                    fixation_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'fixation_offset':
                    # fixation_offset %i at %f
                    fixation_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'response_onset':
                    # response_onset %i at %f
                    response_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'response_offset':
                    # response_offset %i at %f
                    response_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])

            if len(la) > 5:
                if la[2] == 'trial' and la[5]=='LeftArrow':
                    # trial %i event LeftArrow
                    button_press_onset[int(la[3]) -1 + t_run * num_trials] = float(la[1])
                    button_left[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[5]=='RightArrow':
                    # trial %i event RightArrow
                    button_press_onset[int(la[3]) - 1 + t_run *num_trials] = float(la[1])
                    button_right[int(la[3]) - 1 + t_run * num_trials] = float(la[1])

        if first_time and last_time:
            first_last_time = True
            msgfid.close();
    t_run += 1

# create events dataframe
for run_num, run in enumerate(event_filenames):
    df_run = pd.read_csv(run, sep="\t")
    if run_num  > 0 :
        df_events = pd.concat([df_events, df_run])
    else :
        df_events = df_run
        

msg_dict = {}
for msg_output in msg_outputs:
    eval("msg_dict.update({'%s':%s})"%(msg_output,msg_output))

msg_dict.update({'trial_duration': trial_offset-trial_onset})
msg_dict.update({'fix_check_duration': fixation_onset-trial_onset})
msg_dict.update({'fixation_duration': fixation_offset-fixation_onset})
msg_dict.update({'response_duration': response_offset-response_onset})
msg_dict.update({'motion_duration': motion_offset-motion_onset})
msg_dict.update({'reaction_time': button_press_onset-response_onset})

df_msg = pd.DataFrame(msg_dict)
df_all = pd.concat([df_events.reset_index(drop=True),
                    df_msg.reset_index(drop=True)], axis=1)


In [6]:
df_all

,onset,duration,run_number,trial_number,task,ext_mot_pos,ext_mot_ver_dir,staircase_num,fix_off_prct,trial_type,...,fixation_onset,fixation_offset,response_onset,response_offset,trial_duration,fix_check_duration,fixation_duration,response_duration,motion_duration,reaction_time
0,17687.24479,3.724464,1,1,2,1,1,1,2,1,...,15264353.0,15264953.0,15266360.0,15267353.0,3724.0,724.0,600.0,993.0,2001.0,-15266360.0
1,17690.97759,3.691144,1,2,2,2,1,2,1,2,...,15268060.0,15268452.0,15270059.0,15271053.0,3692.0,700.0,392.0,994.0,1993.0,-15270059.0
2,17694.67717,3.507722,1,3,2,1,2,1,1,2,...,15271576.0,15271969.0,15273575.0,15274569.0,3509.0,517.0,393.0,994.0,1993.0,-15273575.0
3,17698.19323,3.507852,1,4,2,1,1,2,5,1,...,15275092.0,15276285.0,15277091.0,15278085.0,3510.0,517.0,1193.0,994.0,1993.0,-15277091.0
4,17701.70948,3.516104,1,5,2,1,2,1,4,1,...,15278617.0,15279610.0,15280616.0,15281613.0,3518.0,525.0,993.0,997.0,1992.0,-15280616.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,18844.79906,3.707801,2,96,2,1,2,1,1,1,...,16421889.0,16422282.0,16423888.0,16424881.0,3709.0,717.0,393.0,993.0,1993.0,-16423888.0
196,18848.51520,3.999430,2,97,2,1,1,1,5,2,...,16425898.0,16427088.0,16427896.0,16428890.0,4001.0,1010.0,1190.0,994.0,1991.0,-16427896.0
197,18852.52293,3.999504,2,98,2,2,1,1,4,1,...,16429906.0,16430896.0,16431904.0,16432898.0,4001.0,1010.0,990.0,994.0,1991.0,-16431904.0
198,18856.53076,3.691158,2,99,2,2,1,1,2,1,...,16433604.0,16434197.0,16435604.0,16436597.0,3693.0,700.0,593.0,993.0,1993.0,-16435604.0


#### Step 2: extract saccades for each trials

In [7]:
# Define parameters
sampling_rate = 1000                   # eyetrakcing sampling rate
velocity_th = 1.5                      # velocity sd threshold
min_dur = 20                           # threshold minimum duration
merge_interval = 20                    # interval between saccade events
cor_sac_onset_th = 100                 # corrective saccade onset threshold (inferior or equal in ms)
fix_area_rad = 2                       # saccade onset position tolerance area in dva
sac_area_rad = 4                       # saccade landing area tolerance in dva
sac_lat_min = -100                     # saccade latency minimum duration
sac_lat_max = 800                      # saccade latency maximum duration

In [8]:
sac_outputs = [ 'miss_time_trial', 'blink_trial', 'no_saccade_trial', 'main_sac_trial', 'innacurate_sac_trial', 'early_sac_trial', 'late_sac_trial',
                'sac_x_onset_trial', 'sac_x_offset_trial', 'sac_y_onset_trial', 'sac_y_offset_trial', 'sac_t_onset_trial', 'sac_t_offset_trial', 
                'sac_dur_trial', 'sac_vpeak_trial', 'sac_dist_trial', 'sac_amp_trial', 'sac_dist_ang_trial', 'sac_lat_trial', 
                'sac_amp_ang_trial', 'cor_sac_trial', 'cor_sac_x_onset_trial', 'cor_sac_x_offset_trial', 'cor_sac_y_onset_trial', 
                'cor_sac_y_offset_trial', 'cor_sac_t_onset_trial', 'cor_sac_t_offset_trial', 'cor_sac_dur_trial', 'cor_sac_vpeak_trial', 
                'cor_sac_dist_trial', 'cor_sac_amp_trial', 'cor_sac_dist_ang_trial', 'cor_sac_amp_ang_trial', 'gabor_sac_onset_x',
                'gabor_sac_onset_y', 'sac_onset_gabor_path_prct']
 
for sac_output in sac_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(sac_output))

print('Saccade extraction:')
t_run = 0
first_fig = 0

for eyetrack_filname, mat_filename in zip(eyetrack_filnames, mat_filenames):
    print(eyetrack_filname)
    
    # load matfile
    mat_dat = scipy.io.loadmat(mat_filename)
    
    scr_x_mid = mat_dat['config']['scr'][0][0][0]['x_mid'][0][0][0]
    scr_y_mid = mat_dat['config']['scr'][0][0][0]['y_mid'][0][0][0]
    frame_duration = mat_dat['config']['scr'][0][0][0]['frame_duration'][0][0][0]
    ppd = mat_dat['config']['const'][0][0][0]['ppd'][0][0][0]
    ecc = mat_dat['config']['const'][0][0][0]['ecc'][0][0][0]
    gabor_path = mat_dat['config']['const'][0][0][0]['gabor_path'][0][0][0]
    ext_motion_steps = mat_dat['config']['const'][0][0][0]['ext_motion_steps'][0][0][0]
    fix_area_rad_pix = ppd * fix_area_rad
    sac_area_rad_pix = ppd * sac_area_rad
    
    # load eyetrack data
    eye_data_run = np.genfromtxt(eyetrack_filname.replace('.edf','.dat'), usecols=(0, 1, 2))

    for trial in np.arange(0, num_trials):
        
        # define trial
        miss_time, blink, no_saccade, main_saccade, corrective_saccade, num_sac = 0, 0, 0, 0, 0, 0
        trial_idx = trial + t_run * num_trials
        df_t = df_all.loc[(df_all.run_number == t_run + 1) & (df_all.trial_number == trial+1)]
        trial_data_logic = np.logical_and(eye_data_run[:,0] >= float(df_t.trial_onset),
                                          eye_data_run[:,0] <= float(df_t.trial_offset))
        blink_data_logic = np.logical_and(eye_data_run[:,0] >= float(df_t.trial_onset),
                                          eye_data_run[:,0] <= float(df_t.fix_break + 200))

        # get stimulus path corrdinates
        if df_t.ext_mot_pos.values == 1: ctr_ext_motion = [scr_x_mid - ecc, scr_y_mid];
        elif df_t.ext_mot_pos.values == 2: ctr_ext_motion = [scr_x_mid + ecc, scr_y_mid];

        if df_t.ext_mot_ver_dir.values == 1: amp_from_ctr = np.linspace(
            gabor_path/2, -gabor_path/2, ext_motion_steps)
        elif df_t.ext_mot_ver_dir.values == 2: amp_from_ctr = np.linspace(
            -gabor_path/2, gabor_path/2, ext_motion_steps)

        gabor_ctrs = [ctr_ext_motion[0] + amp_from_ctr * np.cos(np.deg2rad(df_t.ext_mot_ori.values-90)),
                      ctr_ext_motion[1] + amp_from_ctr * np.sin(np.deg2rad(df_t.ext_mot_ori.values-90))]
        gabor_ctrs_dva = [(gabor_ctrs[0] - scr_x_mid)/ppd,
                          (-1 * (gabor_ctrs[1] - scr_y_mid))/ppd]
        

        # Missing data point detection
        if np.sum(np.diff(eye_data_run[trial_data_logic, 0]) > 1000 / sampling_rate) > 0:
            miss_time = 1
            miss_time_trial[trial_idx] = 1

        # Blink detection
        if not miss_time:
            if np.sum(eye_data_run[blink_data_logic, 1]== -1):
                blink = 1
                blink_trial[trial_idx] = 1

        # Main and corrective saccade detection
        if not miss_time and not blink:
            t, x, y = eye_data_run[trial_data_logic, 0], eye_data_run[trial_data_logic, 1], eye_data_run[trial_data_logic, 2]
            vx, vy = vecvel(x, y, sampling_rate)
            sac = microsacc_merge(x, y, vx, vy, velocity_th, min_dur, merge_interval)
            ms = saccpar(sac)
                
            if np.isnan(ms[0,0]):
                #4 no saccade
                no_saccade = 1
                no_saccade_trial[trial_idx] = 1
 
            if not no_saccade:
                innacurate_sac = 1
                
                # Define fixation 
                fix_pos_x, fix_pos_y = scr_x_mid, scr_y_mid

                # Define saccade posiiton (motion path center position)
                ext_motion_position = int(df_t.ext_mot_pos)
                if ext_motion_position == 1:
                    sac_pos_x = scr_x_mid - ecc
                elif ext_motion_position == 2:
                    sac_pos_x = scr_x_mid + ecc
                sac_pos_y = scr_y_mid
                
                while num_sac < ms.shape[0]:
                    
                    # Main saccade detection
                    fix_cor = isincircle(x[int(ms[num_sac, 0])], y[int(ms[num_sac, 0])], 
                                         fix_pos_x, fix_pos_y, fix_area_rad_pix)
                    sac_cor = isincircle(x[int(ms[num_sac, 1])], y[int(ms[num_sac, 1])], 
                                         sac_pos_x, sac_pos_y, sac_area_rad_pix)

                    if np.logical_and(fix_cor,sac_cor):
                        main_saccade = 1
                        innacurate_sac = 0
                        
                        sac_x_onset_trial[trial_idx] = (x[int(ms[num_sac, 0])] - scr_x_mid) / ppd
                        sac_x_offset_trial[trial_idx] = (x[int(ms[num_sac, 1])] - scr_x_mid) / ppd
                        sac_y_onset_trial[trial_idx] = -1*(y[int(ms[num_sac, 0])] - scr_y_mid) / ppd
                        sac_y_offset_trial[trial_idx] = -1*(y[int(ms[num_sac, 1])] - scr_y_mid) / ppd
                        sac_t_onset_trial[trial_idx] = t[int(ms[num_sac, 0])]
                        sac_t_offset_trial[trial_idx] = t[int(ms[num_sac, 1])]
                        sac_dur_trial[trial_idx] = ms[num_sac, 2]
                        sac_lat_trial[trial_idx] = t[int(ms[num_sac, 0])] - float(df_t.fixation_offset)
                        sac_vpeak_trial[trial_idx] = ms[num_sac, 3] / ppd
                        sac_dist_trial[trial_idx] = ms[num_sac, 4] / ppd
                        sac_amp_trial[trial_idx] = ms[num_sac, 6] / ppd
                        sac_dist_ang_trial[trial_idx] = ms[num_sac, 5]
                        sac_amp_ang_trial[trial_idx] = ms[num_sac, 7]

                        if sac_lat_trial[trial_idx] < sac_lat_min:
                            early_sac_trial[trial_idx] = 1
                            main_saccade = 0
                            
                        if sac_lat_trial[trial_idx] > sac_lat_max:
                            late_sac_trial[trial_idx] = 1
                            main_saccade = 0

                        # get stimulus position at saccade onset
                        if main_saccade:
                            sac_onset_nbf = round((sac_t_onset_trial[trial_idx] -  float(df_t.motion_onset)) 
                                                  / (frame_duration * 1000))
                            
                            gabor_sac_onset_x[trial_idx] = gabor_ctrs_dva[0][sac_onset_nbf]
                            gabor_sac_onset_y[trial_idx] = gabor_ctrs_dva[1][sac_onset_nbf]
                            sac_onset_gabor_path_prct[trial_idx] = sac_onset_nbf/ext_motion_steps
                            
                    # Corrective saccade detection
                    if main_saccade and corrective_saccade == 0:
                        if t[int(ms[num_sac, 0])] <= t[int(ms[num_sac - 1, 0])] + cor_sac_onset_th:
                            corrective_saccade = 1
                            
                            cor_sac_x_onset_trial[trial_idx] = (x[int(ms[num_sac, 0])] - scr_x_mid) / ppd
                            cor_sac_x_offset_trial[trial_idx] = (x[int(ms[num_sac, 1])] - scr_x_mid) / ppd
                            cor_sac_y_onset_trial[trial_idx] = -1*(y[int(ms[num_sac, 0])] - scr_y_mid) / ppd
                            cor_sac_y_offset_trial[trial_idx] = -1*(y[int(ms[num_sac, 1])] - scr_y_mid) / ppd
                            cor_sac_t_onset_trial[trial_idx] = t[int(ms[num_sac, 0])]
                            cor_sac_t_offset_trial[trial_idx] = t[int(ms[num_sac, 1])]
                            cor_sac_dur_trial[trial_idx] = ms[num_sac, 2]
                            cor_sac_vpeak_trial[trial_idx] = ms[num_sac, 3] / ppd
                            cor_sac_dist_trial[trial_idx] = ms[num_sac, 4] / ppd
                            cor_sac_amp_trial[trial_idx] = ms[num_sac, 6] / ppd
                            cor_sac_dist_ang_trial[trial_idx] = ms[num_sac, 5]
                            cor_sac_amp_ang_trial[trial_idx] = ms[num_sac, 7]
                    
                    num_sac += 1 
                    
                    main_sac_trial[trial_idx] = main_saccade
                    cor_sac_trial[trial_idx] = corrective_saccade
                    innacurate_sac_trial[trial_idx] = innacurate_sac    

    t_run +=1
print('Done')

saccade_dict = {}
for sac_output in sac_outputs:
    eval("saccade_dict.update({'%s':%s})"%(sac_output,sac_output))

df_saccade = pd.DataFrame(saccade_dict)
df_all = pd.concat([df_all.reset_index(drop=True),
                    df_saccade.reset_index(drop=True)], axis=1)

for eyetrack_filname in eyetrack_filnames:
    os.remove(eyetrack_filname.replace('.edf','.msg'))
    os.remove(eyetrack_filname.replace('.edf','.dat'))

Saccade extraction:
../data/sub-06/ses-02/beh/sub-06_ses-02_task-DoubleDriftSaccade_run-01_eyetrack.edf
../data/sub-06/ses-02/beh/sub-06_ses-02_task-DoubleDriftSaccade_run-02_eyetrack.edf
Done


In [9]:
# Save dataframe
df_all.to_csv('{}/{}/{}/beh/{}_task-DoubleDriftSaccade_data.csv'.format(data_dir, subject, session, subject), na_rep='n/a')